In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.applications import resnet
from tensorflow.keras.applications import VGG16,InceptionResNetV2,InceptionV3,ResNet50,ResNet152V2,MobileNetV2,ResNet101,Xception,VGG19,MobileNet,DenseNet201,NASNetMobile,DenseNet169,MobileNet
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [ ]:
INIT_LR = 1e-3
EPOCHS = 50
BS = 5

In [ ]:
print("[INFO] loading images...")
imagePaths =list(paths.list_images("/content/drive/MyDrive/3d printing/Experiment2"))
data = []
labels = []

[INFO] loading images...


In [ ]:
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# load the image, swap color channels, and resize it to be a fixed
	# 224x224 pixels while ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 1]
data = np.array(data) / 255.0
labels = np.array(labels)

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, random_state=42)
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=15,
	fill_mode="nearest")

ValueError: y has 0 samples: array([], dtype=float64)

In [ ]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off

baseModel =VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
#baseModel =InceptionResNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
#baseModel =ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
#baseModel =ResNet101(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
#baseModel =MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
#baseModel = VGG19(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False
	# compile our model

In [ ]:
# compile our model
print("[INFO] compiling model...")
#opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
opt = Adam()
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
#model.compile(loss="hinge", optimizer=opt,
	#metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)
#model.save("vgg16xray.h5")

In [ ]:
pip install quiver_engine

In [ ]:
#classification report on training
predIdxs = model.predict(trainX)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
trainpredict = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(trainY.argmax(axis=1), trainpredict))

In [ ]:
# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(trainY.argmax(axis=1), trainpredict)
total = sum(sum(cm))
print(cm)
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
# show the confusion matrix, accuracy, sensitivity, and specificity
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

In [ ]:
from sklearn.metrics import accuracy_score
_, train_acc =model.evaluate(trainX, trainY)

In [ ]:
_, test_acc = model.evaluate(testX, testY)

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdys = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
testpredict = np.argmax(predIdys, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), testpredict,
	target_names=lb.classes_))

In [ ]:
# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testY.argmax(axis=1), testpredict)
total = sum(sum(cm))
print(cm)
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
# show the confusion matrix, accuracy, sensitivity, and specificity
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import roc_curve
fig = plt.figure(figsize = (4, 3))
fpr1,tpr1,_=roc_curve(np.argmax(trainY, axis=1),np.argmax(predIdxs, axis=1))
fpr2,tpr2,_=roc_curve(np.argmax(testY, axis=1),np.argmax(predIdys, axis=1))
area_under_curve1=auc(fpr1,tpr1)
random_probs=[0 for i in range(len(trainY.ravel()))]
p_fpr1,p_tpr1,threshold=roc_curve(trainY.ravel(),random_probs, pos_label=1)
plt.plot(fpr1,tpr1, label='Train AUC = {:.3f}'.format(area_under_curve1))
plt.plot(p_fpr1, p_tpr1)
area_under_curve2=auc(fpr2,tpr2)
random_probs2=[0 for i in range(len(testY.ravel()))]
p_fpr2,p_tpr2,threshold=roc_curve(testY.ravel(),random_probs2, pos_label=1)
plt.plot(fpr2,tpr2, label='Test AUC = {:.3f}'.format(area_under_curve2))
plt.plot(p_fpr2, p_tpr2)
#fpr2,tpr2,_=roc_curve(y_test2.ravel(), yprr2[:,0], pos_label=1)
#fpr3,tpr3,_=roc_curve(y_test3.ravel(), yprr3[:,0], pos_label=1)
#fpr4,tpr4,_=roc_curve(y_test4.ravel(), yprr4[:,0], pos_label=1)
#random_probs2=[0 for i in range(len(y_test2.ravel()))]
#random_probs3=[0 for i in range(len(y_test3.ravel()))]
#random_probs4=[0 for i in range(len(y_test4.ravel()))]
#p_fpr2,p_tpr2,_=roc_curve(y_test2.ravel(),random_probs2, pos_label=1)
#p_fpr3,p_tpr3,_=roc_curve(y_test3.ravel(),random_probs3, pos_label=1)
#p_fpr4,p_tpr4,_=roc_curve(y_test4.ravel(),random_probs4, pos_label=1)
#area_under_curve2 = auc(fpr2, tpr2)
#area_under_curve3 = auc(fpr3, tpr3)
#area_under_curve4 = auc(fpr4, tpr4)
#plt.plot(fpr2,tpr2, label='Borderline-SMOTE= {:.3f}'.format(area_under_curve2))
#plt.plot(p_fpr2, p_tpr2, linestyle='--', color='blue')
#plt.plot(fpr3,tpr3, label='GAN= {:.3f}'.format(area_under_curve3))
#plt.plot(p_fpr3, p_tpr3)
#plt.plot(fpr4,tpr4, label='BSGAN= {:.3f}'.format(area_under_curve4))
#plt.plot(p_fpr4, p_tpr4)

    # x label
plt.xlabel('FPR',fontsize=14, fontdict=dict(weight='bold'))
    # y label
plt.ylabel('TPR', fontsize=14, fontdict=dict(weight='bold'))
plt.xticks( rotation=0, weight = 'bold', )
plt.yticks( rotation=0, weight = 'bold')
plt.tick_params(rotation=0,axis='y', labelsize=12)
plt.tick_params(rotation=0,axis='x', labelsize=12)
plt.legend()
plt.legend(prop={'size':12})
#plt.legend(['without oversampling = {:.3f}'.format(area_under_curve1)])
#plt.legend(['BorderlineSMOTE = {:.3f}'.format(area_under_curve2)],loc='best')
plt.savefig('ROC',dpi=300)
plt.show();

In [ ]:
#import seaborn as sns
#plt.style.use('default')
from matplotlib.ticker import FormatStrFormatter

fig, ax = plt.subplots()
#fig = plt.figure(figsize = (4, 3))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

N=50
fig, ax = plt.subplots()
#plt.rcParams["font.family"] = "serif"
acc = H.history['accuracy']
val_acc = H.history['val_accuracy']
#font={'size':10}
#matplotlib.rc('font',**font)
loss = H.history['loss']
val_loss = H.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(np.arange(0, N), H.history["accuracy"], label="TA")
#plt.plot(epochs, loss, label='Training loss')
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="VA")
plt.plot(epochs, loss, label='TL')
#plt.plot(np.arange(0, N), H.history["val_accuracy"], label="validation accuracy")
plt.plot(np.arange(0, N), val_loss, label='VS')
#plt.plot(epochs, acc, 'bo', label='Training acc')
#plt.plot(1, val_acc, 'b', label='Validation acc')
#plt.title('Training and validation accuracy')
plt.xlabel('Epochs',fontsize=16, fontdict=dict(weight='bold'))
    # y label
plt.ylabel('Accuracy/Loss', fontsize=16, fontdict=dict(weight='bold'))
#plt.ylabel("Accuracy")
#plt.xlabel("Epochs")
plt.xticks( rotation=0, weight = 'bold' )
plt.yticks( rotation=0, weight = 'bold')
plt.tick_params(rotation=0,axis='y', labelsize=14)
plt.tick_params(rotation=0,axis='x', labelsize=14)
#plt.grid('white')
#plt.grid(axis='x', color='0.95')
plt.legend(loc='best')
plt.legend(prop={'size':12})
fig = plt.figure(figsize = (4, 3))
plt.savefig('vgg19acc.png',dpi=1000)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
df=cv2.imread("/content/drive/MyDrive/3d printing/Experiment2/badcylinder/IMG_9078.JPG")
df=cv2.resize(df,(224,224))
cv2_imshow(df)

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from skimage import data, io, filters,transform
import tensorflow

import skimage
#xi = cv2.imread("/content/covid_31.jpg")
#xi = cv2.imread("/content/2019-novel-Coronavirus-severe-adult-respiratory-dist_2020_International-Jour-p3-89%0.png")

Xi = skimage.io.imread("/content/drive/MyDrive/3d printing/Experiment 2/bad/badcylinder/IMG_9087.JPG")
#Xi = skimage.io.imread("/content/drive/MyDrive/monkeypox/augmentedgray/Monekypox_gray_augmented/monkeygray_aug542.jpg")


#xi=cv2.resize(xi,(299,299))
#cv2_imshow(xi/2+0.5)

Xi = skimage.transform.resize(Xi, (224, 224,3))

Xi = (Xi - 0.5)*2 #Inception pre-processing

skimage.io.imshow(Xi/2+0.5) # Show image before inception preprocessing



#Predict class for image using InceptionV3

import keras

from keras.applications.imagenet_utils import decode_predictions

np.random.seed(222)

#inceptionV3_model = tensorflow.keras.applications.inception_v3.InceptionV3() #Load pretrained model
#InceptionResNetV2_model=tensorflow.keras.applications.inception_resnet_v2.InceptionResNetV2()
#ResNet50_model=tensorflow.keras.applications.resnet50.ResNet50()
#ResNet101_model=tensorflow.keras.applications.ResNet101()
#Mobilenetv2_model=tensorflow.keras.applications.mobilenet_v2.MobileNetV2()
#VGG19_model=tensorflow.keras.applications.vgg19.VGG19()
VGG16_model=tensorflow.keras.applications.vgg16.VGG16()

#preds = ResNet101_model.predict(Xi[np.newaxis,:,:,:])
#preds = ResNet50_model.predict(xi[np.newaxis,:,:,:])
#preds = InceptionResNetV2_model.predict(Xi[np.newaxis,:,:,:])
#preds = Mobilenetv2_model.predict(Xi[np.newaxis,:,:,:])
#preds = VGG19_model.predict(Xi[np.newaxis,:,:,:])
preds = VGG16_model.predict(Xi[np.newaxis,:,:,:])


top_pred_classes = preds[0].argsort()[-5:][::-1] # Save ids of top 5 classes

decode_predictions(preds)[0] #Print top 5 classes
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow


import skimage
#xi = cv2.imread("/content/drive/MyDrive/monkeypox/gray/Monkeypox_gray/monkeypox00.jpg")
#xi = cv2.imread("/content/2019-novel-Coronavirus-severe-adult-respiratory-dist_2020_International-Jour-p3-89%0.png")

Xi = skimage.io.imread("/content/drive/MyDrive/3d printing/Experiment 2/bad/badcylinder/IMG_9087.JPG")
#Xi = skimage.io.imread("/content/drive/MyDrive/monkeypox/augmentedgray/Normal_image_gray_augmented/normalgray_aug105.jpg")

#xi=cv2.resize(xi,(299,299))
#cv2_imshow(xi/2+0.5)

Xi = skimage.transform.resize(Xi, (299,299))

Xi = (Xi - 0.5)*2 #Inception pre-processing

skimage.io.imshow(Xi/2+0.5) # Show image before inception preprocessing
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
#Generate segmentation for image

import skimage.segmentation
Xi = skimage.transform.resize(Xi, (299,299,3))
superpixels = skimage.segmentation.quickshift(Xi, kernel_size=4,max_dist=200, ratio=0.2)

num_superpixels = np.unique(superpixels).shape[0]

skimage.io.imshow(skimage.segmentation.mark_boundaries(Xi/2+0.5, superpixels))



#Generate perturbations

num_perturb = 150

perturbations = np.random.binomial(1, 0.5, size=(num_perturb, num_superpixels))



#Create function to apply perturbations to images

import copy

def perturb_image(img,perturbation,segments):

  active_pixels = np.where(perturbation == 1)[0]

  mask = np.zeros(segments.shape)

  for active in active_pixels:

      mask[segments == active] = 1

  perturbed_image = copy.deepcopy(img)

  perturbed_image = perturbed_image*mask[:,:,np.newaxis]

  return perturbed_image



#Show example of perturbations

print(perturbations[0])

skimage.io.imshow(perturb_image(Xi/2+0.5,perturbations[0],superpixels))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
from skimage import io
from tensorflow.keras.preprocessing import image

#url = '/content/drive/MyDrive/monkeypox/gray/Normal_image_gray/normal00.jpg'
url="/content/drive/MyDrive/3d printing/Experiment 2/bad/badcylinder/IMG_9087.JPG"
#url="/content/drive/MyDrive/monkeypox/augmentedgray/Monekypox_gray_augmented/monkeygray_aug542.jpg"
#url="/content/drive/MyDrive/monkeypox/augmentedgray/Normal_image_gray_augmented/normalgray_aug105.jpg"

def read_and_transform_img(url):

    img = skimage.io.imread(url)
    img = skimage.transform.resize(img, (224, 224,3))

    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    return img

images = read_and_transform_img(url)

preds = model.predict(images)
prediction = np.argmax(preds)
pct = np.max(preds)

if prediction == 0:
    print('It\'s a good!')
elif prediction == 1:
    print('It\'s a bad!')
else:
    print('Unable to predict!')

print(pct)

In [ ]:
pip install lime

In [ ]:
from lime import lime_image

explainer = lime_image.LimeImageExplainer()

In [ ]:
explanation = explainer.explain_instance(images[0].astype('double'), model.predict,
                                         top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
from skimage.segmentation import mark_boundaries

temp_1, mask_1 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5,5))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax1.axis('off')
ax2.axis('off')

In [ ]:
from tensorflow.keras.applications import ResNet50
res_model = ResNet50()
res_model.summary()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
#img = cv2.imread('/content/drive/MyDrive/3d printing/Experiment 2/goodcylinder/IMG_8935.JPG')
img = cv2.imread('/content/drive/MyDrive/3d printing/Experiment 2/bad/badcylinder/IMG_9087.JPG')
#img = cv2.imread("/content/drive/MyDrive/monkeypox/augmentedgray/Monekypox_gray_augmented/monkeygray_aug542.jpg")
img = cv2.resize(img, (224, 224))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
X = np.expand_dims(img, axis=0).astype(np.float32)
X = preprocess_input(X)

In [ ]:
from tensorflow.keras.models import Model
conv_output = res_model.get_layer("conv5_block3_out").output
pred_ouptut = res_model.get_layer("predictions").output
model = Model(res_model.input, outputs=[conv_output, pred_ouptut])

In [ ]:
conv, pred = model.predict(X)
decode_predictions(pred)

In [ ]:
scale = 224 / 7
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.xticks( rotation=0, weight = 'bold' )
    plt.yticks( rotation=0, weight = 'bold')
    plt.tick_params(rotation=0,axis='y', labelsize=12)
    plt.tick_params(rotation=0,axis='x', labelsize=12)
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    #plt.imshow(zoom(conv[0, :,:,i], zoom=(scale, scale)))
    plt.imshow(zoom(conv[0, :,:,i], zoom=(scale, scale)), cmap='jet', alpha=0.3)

In [ ]:
target = np.argmax(pred, axis=1).squeeze()
w, b = model.get_layer("predictions").weights
weights = w[:, target].numpy()
heatmap = conv.squeeze() @ weights

In [ ]:
scale = 224 / 7
plt.figure(figsize=(5, 5))
plt.xticks( rotation=0, weight = 'bold' )
plt.yticks( rotation=0, weight = 'bold')
plt.tick_params(rotation=0,axis='y', labelsize=14)
plt.tick_params(rotation=0,axis='x', labelsize=14)
plt.imshow(img)
plt.imshow(zoom(heatmap, zoom=(scale, scale)), cmap='jet', alpha=0.5)

In [ ]:
!pip install gradcam

In [ ]:
import cv2
import torch

layer_finders = {}


def register_layer_finder(model_type):
    def register(func):
        layer_finders[model_type] = func
        return func
    return register


def visualize_cam(mask, img, alpha=1.0):
    """Make heatmap from mask and synthesize GradCAM result image using heatmap and img.
    Args:
        mask (torch.tensor): mask shape of (1, 1, H, W) and each element has value in range [0, 1]
        img (torch.tensor): img shape of (1, 3, H, W) and each pixel value is in range [0, 1]
    Return:
        heatmap (torch.tensor): heatmap img shape of (3, H, W)
        result (torch.tensor): synthesized GradCAM result of same shape with heatmap.
    """
    heatmap = (255 * mask.squeeze()).type(torch.uint8).cpu().numpy()
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = torch.from_numpy(heatmap).permute(2, 0, 1).float().div(255)
    b, g, r = heatmap.split(1)
    heatmap = torch.cat([r, g, b]) * alpha

    result = heatmap+img.cpu()
    result = result.div(result.max()).squeeze()

    return heatmap, result


@register_layer_finder('resnet')
def find_resnet_layer(arch, target_layer_name):
    """Find resnet layer to calculate GradCAM and GradCAM++
    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'conv1'
            target_layer_name = 'layer1'
            target_layer_name = 'layer1_basicblock0'
            target_layer_name = 'layer1_basicblock0_relu'
            target_layer_name = 'layer1_bottleneck0'
            target_layer_name = 'layer1_bottleneck0_conv1'
            target_layer_name = 'layer1_bottleneck0_downsample'
            target_layer_name = 'layer1_bottleneck0_downsample_0'
            target_layer_name = 'avgpool'
            target_layer_name = 'fc'
    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """
    if 'layer' in target_layer_name:
        hierarchy = target_layer_name.split('_')
        layer_num = int(hierarchy[0].lstrip('layer'))
        if layer_num == 1:
            target_layer = arch.layer1
        elif layer_num == 2:
            target_layer = arch.layer2
        elif layer_num == 3:
            target_layer = arch.layer3
        elif layer_num == 4:
            target_layer = arch.layer4
        else:
            raise ValueError('unknown layer : {}'.format(target_layer_name))

        if len(hierarchy) >= 2:
            bottleneck_num = int(hierarchy[1].lower().lstrip('bottleneck').lstrip('basicblock'))
            target_layer = target_layer[bottleneck_num]

        if len(hierarchy) >= 3:
            target_layer = target_layer._modules[hierarchy[2]]

        if len(hierarchy) == 4:
            target_layer = target_layer._modules[hierarchy[3]]

    else:
        target_layer = arch._modules[target_layer_name]

    return target_layer


@register_layer_finder('densenet')
def find_densenet_layer(arch, target_layer_name):
    """Find densenet layer to calculate GradCAM and GradCAM++
    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'features'
            target_layer_name = 'features_transition1'
            target_layer_name = 'features_transition1_norm'
            target_layer_name = 'features_denseblock2_denselayer12'
            target_layer_name = 'features_denseblock2_denselayer12_norm1'
            target_layer_name = 'features_denseblock2_denselayer12_norm1'
            target_layer_name = 'classifier'
    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """

    hierarchy = target_layer_name.split('_')
    target_layer = arch._modules[hierarchy[0]]

    if len(hierarchy) >= 2:
        target_layer = target_layer._modules[hierarchy[1]]

    if len(hierarchy) >= 3:
        target_layer = target_layer._modules[hierarchy[2]]

    if len(hierarchy) == 4:
        target_layer = target_layer._modules[hierarchy[3]]

    return target_layer


@register_layer_finder('vgg')
def find_vgg_layer(arch, target_layer_name):
    """Find vgg layer to calculate GradCAM and GradCAM++
    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'features'
            target_layer_name = 'features_42'
            target_layer_name = 'classifier'
            target_layer_name = 'classifier_0'
    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """
    hierarchy = target_layer_name.split('_')

    if len(hierarchy) >= 1:
        target_layer = arch.features

    if len(hierarchy) == 2:
        target_layer = target_layer[int(hierarchy[1])]

    return target_layer


@register_layer_finder('alexnet')
def find_alexnet_layer(arch, target_layer_name):
    """Find alexnet layer to calculate GradCAM and GradCAM++
    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'features'
            target_layer_name = 'features_0'
            target_layer_name = 'classifier'
            target_layer_name = 'classifier_0'
    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """
    hierarchy = target_layer_name.split('_')

    if len(hierarchy) >= 1:
        target_layer = arch.features

    if len(hierarchy) == 2:
        target_layer = target_layer[int(hierarchy[1])]

    return target_layer


@register_layer_finder('squeezenet')
def find_squeezenet_layer(arch, target_layer_name):
    """Find squeezenet layer to calculate GradCAM and GradCAM++
    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'features_12'
            target_layer_name = 'features_12_expand3x3'
            target_layer_name = 'features_12_expand3x3_activation'
    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """
    hierarchy = target_layer_name.split('_')
    target_layer = arch._modules[hierarchy[0]]

    if len(hierarchy) >= 2:
        target_layer = target_layer._modules[hierarchy[1]]

    if len(hierarchy) == 3:
        target_layer = target_layer._modules[hierarchy[2]]

    elif len(hierarchy) == 4:
        target_layer = target_layer._modules[hierarchy[2]+'_'+hierarchy[3]]

    return target_layer


def denormalize(tensor, mean, std):
    if not tensor.ndimension() == 4:
        raise TypeError('tensor should be 4D')

    mean = torch.FloatTensor(mean).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)
    std = torch.FloatTensor(std).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)

    return tensor.mul(std).add(mean)


def normalize(tensor, mean, std):
    if not tensor.ndimension() == 4:
        raise TypeError('tensor should be 4D')

    mean = torch.FloatTensor(mean).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)
    std = torch.FloatTensor(std).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)

    return tensor.sub(mean).div(std)


class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return self.do(tensor)

    def do(self, tensor):
        return normalize(tensor, self.mean, self.std)

    def undo(self, tensor):
        return denormalize(tensor, self.mean, self.std)

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
import torch
import torch.nn.functional as F

#from .utils import layer_finders


class GradCAM:
    """Calculate GradCAM salinecy map.
    Args:
        input: input image with shape of (1, 3, H, W)
        class_idx (int): class index for calculating GradCAM.
                If not specified, the class index that makes the highest model prediction score will be used.
    Return:
        mask: saliency map of the same spatial dimension with input
        logit: model output
    A simple example:
        # initialize a model, model_dict and gradcam
        resnet = torchvision.models.resnet101(pretrained=True)
        resnet.eval()
        gradcam = GradCAM.from_config(model_type='resnet', arch=resnet, layer_name='layer4')
        # get an image and normalize with mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
        img = load_img()
        normed_img = normalizer(img)
        # get a GradCAM saliency map on the class index 10.
        mask, logit = gradcam(normed_img, class_idx=10)
        # make heatmap from mask and synthesize saliency map using heatmap and img
        heatmap, cam_result = visualize_cam(mask, img)
    """

    def __init__(self, arch: torch.nn.Module, target_layer: torch.nn.Module):
        self.model_arch = arch

        self.gradients = dict()
        self.activations = dict()

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]

        def forward_hook(module, input, output):
            self.activations['value'] = output

        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

    @classmethod
    def from_config(cls, arch: torch.nn.Module, model_type: str, layer_name: str):
        target_layer = layer_finders[model_type](arch, layer_name)
        return cls(arch, target_layer)

    def saliency_map_size(self, *input_size):
        device = next(self.model_arch.parameters()).device
        self.model_arch(torch.zeros(1, 3, *input_size, device=device))
        return self.activations['value'].shape[2:]

    def forward(self, input, class_idx=None, retain_graph=False):
        b, c, h, w = input.size()

        logit = self.model_arch(input)
        if class_idx is None:
            score = logit[:, logit.max(1)[-1]].squeeze()
        else:
            score = logit[:, class_idx].squeeze()

        self.model_arch.zero_grad()
        score.backward(retain_graph=retain_graph)
        gradients = self.gradients['value']
        activations = self.activations['value']
        b, k, u, v = gradients.size()

        alpha = gradients.view(b, k, -1).mean(2)
        # alpha = F.relu(gradients.view(b, k, -1)).mean(2)
        weights = alpha.view(b, k, 1, 1)

        saliency_map = (weights*activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.upsample(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data

        return saliency_map, logit

    def __call__(self, input, class_idx=None, retain_graph=False):
        return self.forward(input, class_idx, retain_graph)


class GradCAMpp(GradCAM):
    """Calculate GradCAM++ salinecy map.
    Args:
        input: input image with shape of (1, 3, H, W)
        class_idx (int): class index for calculating GradCAM.
                If not specified, the class index that makes the highest model prediction score will be used.
    Return:
        mask: saliency map of the same spatial dimension with input
        logit: model output
    A simple example:
        # initialize a model, model_dict and gradcampp
        resnet = torchvision.models.resnet101(pretrained=True)
        resnet.eval()
        gradcampp = GradCAMpp.from_config(model_type='resnet', arch=resnet, layer_name='layer4')
        # get an image and normalize with mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
        img = load_img()
        normed_img = normalizer(img)
        # get a GradCAM saliency map on the class index 10.
        mask, logit = gradcampp(normed_img, class_idx=10)
        # make heatmap from mask and synthesize saliency map using heatmap and img
        heatmap, cam_result = visualize_cam(mask, img)
    """

    def forward(self, input, class_idx=None, retain_graph=False):
        b, c, h, w = input.size()

        logit = self.model_arch(input)
        if class_idx is None:
            score = logit[:, logit.max(1)[-1]].squeeze()
        else:
            score = logit[:, class_idx].squeeze()

        self.model_arch.zero_grad()
        score.backward(retain_graph=retain_graph)
        gradients = self.gradients['value']  # dS/dA
        activations = self.activations['value']  # A
        b, k, u, v = gradients.size()

        alpha_num = gradients.pow(2)
        alpha_denom = alpha_num.mul(2) + activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))

        alpha = alpha_num.div(alpha_denom+1e-7)
        positive_gradients = F.relu(score.exp()*gradients)  # ReLU(dY/dA) == ReLU(exp(S)*dS/dA))
        weights = (alpha*positive_gradients).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        saliency_map = (weights*activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.upsample(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map-saliency_map_min).div(saliency_map_max-saliency_map_min).data

        return saliency_map, logit

In [ ]:
import os
import PIL
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torchvision.utils import make_grid, save_image
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
img_dir = '/content/drive/MyDrive/3d printing/Experiment 2/bad/badcylinder'
#img_dir = '/content/drive/MyDrive/monkeypox/augmentedgray/Monekypox_gray_augmented'
img_dir = '/content'
# img_name = 'collies.JPG'
# img_name = 'multiple_dogs.jpg'
# img_name = 'snake.JPEG'
img_name = '34.jfif'
img_path = os.path.join(img_dir, img_name)

pil_img = PIL.Image.open(img_path)
pil_img

In [ ]:
torch_img = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])(pil_img).to(device)
normed_torch_img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(torch_img)[None]

In [ ]:
vgg = models.vgg16(pretrained=True)

In [ ]:
configs = [  dict(model_type='vgg', arch=vgg, layer_name='features_29')]

In [ ]:
alexnet = models.alexnet(pretrained=True)
vgg = models.vgg16(pretrained=True)
resnet = models.resnet101(pretrained=True)
densenet = models.densenet161(pretrained=True)
squeezenet = models.squeezenet1_1(pretrained=True)

In [ ]:
configs = [
    #dict(model_type='alexnet', arch=alexnet, layer_name='features_11'),
    dict(model_type='vgg', arch=vgg, layer_name='features_29'),
    #dict(model_type='resnet', arch=resnet, layer_name='layer4'),
    #dict(model_type='densenet', arch=densenet, layer_name='features_norm5'),
    #dict(model_type='squeezenet', arch=squeezenet, layer_name='features_12_expand3x3_activation')
]

In [ ]:
for config in configs:
    config['arch'].to(device).eval()

cams = [
    [cls.from_config(**config) for cls in (GradCAM, GradCAMpp)]
    for config in configs
]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(2, 2))
#plt.xticks( rotation=0, weight = 'bold' )
#plt.yticks( rotation=0, weight = 'bold')
#plt.tick_params(rotation=0,axis='y', labelsize=14)
#plt.tick_params(rotation=0,axis='x', labelsize=14)
#plt.imshow(img)
images = []
for gradcam, gradcam_pp in cams:
    mask, _ = gradcam(normed_torch_img)
    heatmap, result = visualize_cam(mask, torch_img)

    mask_pp, _ = gradcam_pp(normed_torch_img)
    heatmap_pp, result_pp = visualize_cam(mask_pp, torch_img)

    images.extend([torch_img.cpu(), heatmap, heatmap_pp, result, result_pp])


grid_image = make_grid(images, nrow=5)

In [ ]:
transforms.ToPILImage()(grid_image)